Neste colab você irá implementar um Neurônio Artificial. Ao invés de usarmos o modelo de limiar, vamos implementar o neurônio com potencial de ativação linear.

In [1]:
# Execute esta celula para carregar as funções, mas não se preocupe com o conteudo
# São apenas funções auxiliares para os testes automatizados

def testa_float(esperado:float, obtido:float, tol=1E-9) -> None:
  """
  Compara se a diferenca entre 2 floats esta dentro da tolerancia e imprime
  a mensagem correspondente
  """
  if abs(esperado - obtido) < tol:
    print("Passou no teste")
  else:
    print(f"Erro. Esperado: {esperado}, obtido: {obtido}")


def testa_int(esperado:int, obtido:int):
  if esperado == obtido:
    print("Passou no teste")
  else:
    print(f"Erro. Esperado: {esperado}, obtido: {obtido}")


Implemente o neurônio linear na função abaixo e teste-o na célula seguinte. Para facilitar, o neurônio tem apenas uma entrada, o peso correspondente e o viés (bias).

In [2]:
def neuronio_linear(w1:float, bias:float, x1: float) -> float:
  """
  Implementa um neuronio linear com 1 entrada, 1 pesos e 1 bias
  :param w1: peso aplicado ao sinal de entrada 
  :param bias: viés (bias / deslocamento) do neuronio
  :param x1: 1o sinal de entrada do neuronio
  :return: sinal de saida do neuronio
  """
  # inicio: implemente o codigo que faça o neuronio computar e retornar peso*entrada + bias 
  return (w1 * x1) + bias
  # fim


# --- a chamada abaixo executa a funcao com a entrada especificada e mostra a saida
neuronio_linear(w1=1, bias=0, x1=1)

1

In [3]:
# NAO MUDE O CONTEUDO DESSA CELULA, apenas execute-a para testar a celula anterior
def testa_neuronio_linear():
  print("Teste do neurônio linear")
  # teste 1 
  testa_int(3, neuronio_linear(w1=1, bias=0, x1=3))
  # teste 2 
  testa_int(-1, neuronio_linear(w1=-1, bias=1, x1=2))

  
#### --- a chamada abaixo executa o teste
testa_neuronio_linear()

Teste do neurônio linear
Passou no teste
Passou no teste


Abaixo, encontre o peso e o bias que façam com que seu neurônio "implemente" a função que acerta o conjunto de dados definido.

In [4]:
def testa_funcao_linear():
  """
  Encontre os pesos e o bias que façam o neurônio implementar a função definida no conjunto de pontos abaixo.
  Seu neuronio deve acertar a saida para todas as entradas.
  Este teste assume que sua funcao "neuronio_linear" está funcionando corretamente.
  x1  | saida
  0   |  -2
  0.5 |  0
  1   |  2 
  2   |  6 


  A mensagem "Passou no teste" deve aparecer 4 vezes. 
  Caso nao seja existam pesos e limiares que implementem a função, coloque None nos pesos e no limiar.
  A funcao testa_int vai reclamar, mas saberemos o que aconteceu.
  
  Caso gaste mais de 10 minutos tentando encontrar pesos/limiares sem sucesso, é provavel que não existam mesmo.
  """

  w1 = 4        # ajuste esse valor
  bias = -2      # ajuste esse valor

  # nao mexa daqui pra baixo

  print("x1  | saida")
  for x1, saida in [(0, -2), (0.5, 0), (1, 2), (2, 6)]:
    resposta = neuronio_linear(w1, bias, x1)
    print(f"{x1:.1f}  |    {resposta:.5f}", sep=" ")
    testa_int(saida, resposta)



#### --- a chamada abaixo executa o teste
testa_funcao_linear()

x1  | saida
0.0  |    -2.00000
Passou no teste
0.5  |    0.00000
Passou no teste
1.0  |    2.00000
Passou no teste
2.0  |    6.00000
Passou no teste


**Ajuste automatico de pesos e bias**

Nos próximos passos, você irá implementar o método da descida do gradiente para ajuste automático dos pesos e bias do seu neurônio. Primeiramente, implemente o cálculo do erro. Ele será útil para a atualização por descida do gradiente.

In [5]:
def erro(w:float, b:float, x:float, y:float) -> float:
  """
  Calcula o erro entre o que é predito pelo neuronio linear e 
  o valor alvo fornecido (y)
  :param w: peso para o neuronio linear
  :param b: bias para o neuronio linear
  :param x: sinal de entrada para o neuronio
  :param y: alvo (valor desejado na saída)
  """
  #inicio: seu codigo aqui (dica: chame sua funcao neuronio_linear para fazer a predicao)
  erro = neuronio_linear(w, b, x) - y
  return erro
  #fim

In [6]:
def testa_erro():
  print("Teste da função que calcula o erro de uma predição. A msg 'Passou no teste' tem q aparecer 3 vezes")
  testa_float(-1, erro(w=1, b=1, x=1, y=3))
  testa_float(0, erro(w=1, b=1, x=3, y=4))
  testa_float(1, erro(w=1, b=2, x=4, y=5))

# a chamada abaixo executa o teste
testa_erro()

Teste da função que calcula o erro de uma predição. A msg 'Passou no teste' tem q aparecer 3 vezes
Passou no teste
Passou no teste
Passou no teste


Implementação do calculo do Erro Quadratico Medio e seu teste na celula posterior.

In [7]:
import numpy as np

# considere o conjunto de dados sintetico abaixo
# entradas (x) na primeira coluna, saida desejada (y) na segunda
dataset = np.array([
    [1, 3],
    [2, 4],
    [3, 4], 
    [4, 2]
])

def eqm(w:float, b:float, dados:np.array):
  """
  Calcula o erro quadratico medio das predicoes do neuronio linear
  em um conjunto de dados. Dica: use sua funcao 'erro' que calcula 
  o erro de uma predicao.
  
  :param w: peso para o neuronio (inclinacao da reta)
  :param b: bias do neuronio (intercepto da reta)
  :param dados: matriz com o conjunto de dados, x na coluna 0 e y na coluna 1
  :return: float: o erro quadratico medio
  """
  erro_quad = 0
  # inicio
  tamanho = len(dados)  
  for i in range(tamanho):
    # print(i)
    x = dados[i][0]
    alvo = dados[i][1]
    erro_quad = erro_quad + erro(w, b, x, alvo)**2
  
  # Retorna o erro quadrático médio
  return erro_quad / len(dados)

In [8]:
# o codigo abaixo testa o calculo do EQM
# usamos o dataset definido na celula acima
print("Teste do Erro Quadratico Médio (EQM)")
testa_float(2.75, eqm(w=1, b=1, dados=dataset))

Teste do Erro Quadratico Médio (EQM)
Passou no teste


Agora, preencha as funções que calculam o gradiente do erro quadradico medio (eqm) para os valores de w e b especificados. Cada função calcula uma componente do gradiente (a derivada parcial com relação a w ou b). Teste-os na celula posterior.

In [9]:
def deriv_b(b:float, w:float, dados:np.array) -> float:
  """
  Implemente o calculo da derivada do EQM com relacao ao bias (intercepto da reta)
  :param b: bias (intercepto da reta)
  :param w: peso (inclinacao da reta)
  :param dados: matriz com o conjunto de dados, x na coluna 0 e y na coluna 1
  """
  # inicio
  somatorio_erro = 0
  for i in range(len(dados)):
    # print(i)
    x = dados[i][0]
    alvo = dados[i][1]
    somatorio_erro = somatorio_erro + erro(w, b, x, alvo)


  return (2/len(dados)) * somatorio_erro
  # fim

def deriv_w(b:float, w:float, dados:np.array) -> float:
  """
  Implemente o calculo da derivada do EQM com relacao ao peso w (inclinacao da reta)
  :param b: bias (intercepto da reta)
  :param w: peso (inclinacao da reta)
  :param dados: matriz com o conjunto de dados, x na coluna 0 e y na coluna 1
  """
  # inicio
  somatorio_erro = 0
  for i in range(len(dados)):
    # print(i)
    x = dados[i][0]
    alvo = dados[i][1]
    somatorio_erro = somatorio_erro + (erro(w, b, x, alvo) * x)


  return (2/len(dados)) * somatorio_erro
  # fim

In [10]:
print("Parametros iniciais: b=1, w=1. O dataset é o mesmo da celula do EQM.")
print("Teste da derivada de b:")
testa_float(0.5, deriv_b(b=1, w=1, dados=dataset))

print("Teste da derivada de w:")
testa_float(4.5, deriv_w(b=1, w=1, dados=dataset))


Parametros iniciais: b=1, w=1. O dataset é o mesmo da celula do EQM.
Teste da derivada de b:
Passou no teste
Teste da derivada de w:
Passou no teste


Em seguida preencha as funções que implementam a atualização do peso e bias. O teste é na célula posterior.

In [11]:
def update_w(b:float, w:float, alpha:float, dados:np.array):
  """
  Executa um passo da atualizacao do peso (w) de acordo com a regra de atualizacao
  dada pela descida do gradiente (w_novo = w_antigo - alpha * derivada do erro c/ relacao a w)
  Dica: use a sua implementacao do calculo da derivada
  :param b: bias (intercepto da reta)
  :param w: peso (inclinacao da reta)
  :param dados: matriz com o conjunto de dados, x na coluna 0 e y na coluna 1
  """
  # inicio
  w_novo = w -alpha * deriv_w(b, w, dados)
  return w_novo
  # fim

def update_b(b:float, w:float, alpha:float, dados:np.array):
  """
  Executa um passo da atualizacao do bias (b) de acordo com a regra de atualizacao
  dada pela descida do gradiente (b_novo = b_antigo - alpha * derivada do erro c/ relacao a b)
  Dica: use a sua implementacao do calculo da derivada
  :param b: bias (intercepto da reta)
  :param w: peso (inclinacao da reta)
  :param dados: matriz com o conjunto de dados, x na coluna 0 e y na coluna 1
  """
  # inicio
  b_novo = b -alpha * deriv_b(b, w, dados)
  return b_novo
  # fim


In [12]:
print("Parametros: b=1, w=1, alpha=0.1. O dataset é o mesmo da celula do EQM.")
print("Teste da atualizacao de b:")
b_novo = update_b(b=1, w=1, alpha=0.1, dados=dataset)
testa_float(0.95, b_novo)

print("Teste da atualizacao de w:")
w_novo = update_w(b=1, w=1, alpha=0.1, dados=dataset)
testa_float(0.55, w_novo)
  

Parametros: b=1, w=1, alpha=0.1. O dataset é o mesmo da celula do EQM.
Teste da atualizacao de b:
Passou no teste
Teste da atualizacao de w:
Passou no teste


**Fim do exercício.**